In [1]:
import pandas as pd
import zstandard as zstd
import orjson

In [2]:
_zstd_dctx = zstd.ZstdDecompressor()

In [8]:
json_pth = '../intermediates/uniprot_release_2020_03/parsed_jsons/Q9Y3F1.json.zst'
entry_d = orjson.loads(_zstd_dctx.decompress(open(json_pth, 'rb').read()))

In [9]:
uniprot_acc = entry_d['accession'][0]
uniprot_name = entry_d['name'][0]
protein_name = entry_d['protein']['recommendedName']['fullName']
if not isinstance(protein_name, str):
    protein_name = protein_name['$']

gene_name = entry_d['gene'][0]['name'][0]['$']

KeyError: 'gene'

In [5]:
hgnc_d = next(d for d in entry_d['dbReference'] if d['@type'] == 'HGNC')

In [6]:
hgnc_d

{'@type': 'HGNC',
 '@id': 'HGNC:5661',
 'property': [{'@type': 'gene designation', '@value': 'IGHV5-10-1'}]}

In [7]:
hgnc_gene_id = hgnc_d['@id']
hgnc_gene_name = next(
    (d['@value'] for d in hgnc_d['property'] if d['@type'] == 'gene designation'),
    None
)

In [8]:
seq_d = entry_d['sequence']
seq_d

{'@length': 117,
 '@mass': 12773,
 '@checksum': 'CE54AEC68F01615A',
 '@modified': '2015-10-14',
 '@version': 1,
 '@precursor': True,
 '$': 'MGSTAILALLLAVLQGVCAEVQLVQSGAEVKKPGESLRISCKGSGYSFTSYWISWVRQMPGKGLEWMGRIDPSDSYTNYSPSFQGHVTISADKSISTAYLQWSSLKASDTAMYYCAR'}

In [9]:
out_d = {
    'uniprot_acc': uniprot_acc,
    'uniprot_name': uniprot_name,
    'uniprot_protein_name': protein_name,
    'uniprot_gene_name': gene_name,
    'hgnc_gene_id': hgnc_gene_id,
    'hgnc_gene_name': hgnc_gene_name,
    'sequence_modified_date': seq_d['@modified'],
    'sequence_crc64_checksum': seq_d['@checksum'],
    'sequence': seq_d['$']
}

In [10]:
out_d

{'uniprot_acc': 'A0A0J9YXX1',
 'uniprot_name': 'HV5X1_HUMAN',
 'uniprot_protein_name': 'Immunoglobulin heavy variable 5-10-1',
 'uniprot_gene_name': 'IGHV5-10-1',
 'hgnc_gene_id': 'HGNC:5661',
 'hgnc_gene_name': 'IGHV5-10-1',
 'sequence_modified_date': '2015-10-14',
 'sequence_crc64_checksum': 'CE54AEC68F01615A',
 'sequence': 'MGSTAILALLLAVLQGVCAEVQLVQSGAEVKKPGESLRISCKGSGYSFTSYWISWVRQMPGKGLEWMGRIDPSDSYTNYSPSFQGHVTISADKSISTAYLQWSSLKASDTAMYYCAR'}

In [11]:
df = pd.DataFrame.from_records(
    [out_d],
    columns=['uniprot_acc', 'uniprot_name', 'uniprot_protein_name',
       'uniprot_gene_name', 'hgnc_gene_id', 'hgnc_gene_name',
       'sequence_modified_date', 'sequence_crc64_checksum', 'sequence'])

In [12]:
df

,uniprot_acc,uniprot_name,uniprot_protein_name,uniprot_gene_name,hgnc_gene_id,hgnc_gene_name,sequence_modified_date,sequence_crc64_checksum,sequence
0,A0A0J9YXX1,HV5X1_HUMAN,Immunoglobulin heavy variable 5-10-1,IGHV5-10-1,HGNC:5661,IGHV5-10-1,2015-10-14,CE54AEC68F01615A,MGSTAILALLLAVLQGVCAEVQLVQSGAEVKKPGESLRISCKGSGY...
